In [4]:
import pandas as pd

df = pd.read_pickle('./../DF.pkl')

In [25]:
df.shape

(2830743, 79)

In [20]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [23]:
df[' Label'].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed'], dtype=object)

In [28]:
df.isnull().any().any()

True

In [30]:
df_clean = df.dropna()
df_clean.shape

(2829385, 79)

In [33]:
df_clean[' Label'].value_counts()

BENIGN                        2272688
DoS Hulk                       230124
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name:  Label, dtype: int64

In [39]:
df_clean['Class'] = df_clean[' Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)

C:\Users\tyagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
df_clean.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Class
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN,0
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN,0
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN,0
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN,0
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN,0


In [41]:
df_clean['Class'].value_counts()

0    2272688
1     556697
Name: Class, dtype: int64

In [85]:
df_clean = df_clean[df_clean[' Flow Packets/s'] != 'Infinity']
df_clean[' Flow Packets/s']= pd.to_numeric(df_clean[' Flow Packets/s'])

df_clean = df_clean[df_clean['Flow Bytes/s'] != 'Infinity']
df_clean['Flow Bytes/s']= pd.to_numeric(df_clean['Flow Bytes/s'])

In [86]:
corr_matrix = df_clean.corr()
print(corr_matrix["Class"].sort_values(ascending=False))
y = df_clean['Class']
X = df_clean.drop(['Class', ' Label'], axis=1)

Class                          1.000000
 Bwd Packet Length Std         0.510884
Bwd Packet Length Max          0.492690
 Bwd Packet Length Mean        0.484904
 Avg Bwd Segment Size          0.484904
 Packet Length Std             0.470928
 Max Packet Length             0.454715
 Packet Length Variance        0.454397
 Fwd IAT Std                   0.423303
 Packet Length Mean            0.414750
 Average Packet Size           0.413740
 Idle Max                      0.394752
Idle Mean                      0.390996
 Flow IAT Max                  0.389205
 Fwd IAT Max                   0.389176
 Idle Min                      0.381160
 Flow IAT Std                  0.337203
Fwd IAT Total                  0.215876
 Flow Duration                 0.214274
FIN Flag Count                 0.189587
 PSH Flag Count                0.177193
 Flow IAT Mean                 0.174783
 Bwd IAT Std                   0.158856
 Fwd IAT Mean                  0.150725
 ACK Flag Count                0.125135


In [87]:
import numpy as np
df_Norm = df_clean.drop([' Label'], axis=1)
# Number of data points in the minority class
number_records_fraud = len(df_Norm[df_Norm.Class == 1])
fraud_indices = np.array(df_Norm[df_Norm.Class == 1].index)

# Picking the indices of the normal classes
normal_indices = df_Norm[df_Norm.Class == 0].index

# Out of the indices we picked, randomly select "x" number (number_records_fraud)
random_normal_indices = np.random.choice(normal_indices, number_records_fraud, replace = False)
random_normal_indices = np.array(random_normal_indices)

# Appending the 2 indices
under_sample_indices = np.concatenate([fraud_indices,random_normal_indices])

# Under sample dataset
under_sample_data = df_Norm.iloc[under_sample_indices,:]

X_undersample = under_sample_data.iloc[:, under_sample_data.columns != 'Class']
y_undersample = under_sample_data.iloc[:, under_sample_data.columns == 'Class']

# Showing ratio
print("Percentage of Benign Cases: ", len(under_sample_data[under_sample_data.Class == 0])/len(under_sample_data))
print("Percentage of Malicious Activity: ", len(under_sample_data[under_sample_data.Class == 1])/len(under_sample_data))
print("Total number of cases: ", len(under_sample_data))


MemoryError: 

In [84]:
X_undersample.dtypes

 Destination Port                 int64
 Flow Duration                    int64
 Total Fwd Packets                int64
 Total Backward Packets           int64
Total Length of Fwd Packets       int64
 Total Length of Bwd Packets      int64
 Fwd Packet Length Max            int64
 Fwd Packet Length Min            int64
 Fwd Packet Length Mean         float64
 Fwd Packet Length Std          float64
Bwd Packet Length Max             int64
 Bwd Packet Length Min            int64
 Bwd Packet Length Mean         float64
 Bwd Packet Length Std          float64
Flow Bytes/s                    float64
 Flow Packets/s                 float64
 Flow IAT Mean                  float64
 Flow IAT Std                   float64
 Flow IAT Max                     int64
 Flow IAT Min                     int64
Fwd IAT Total                     int64
 Fwd IAT Mean                   float64
 Fwd IAT Std                    float64
 Fwd IAT Max                      int64
 Fwd IAT Min                      int64


In [ ]:
from sklearn import preprocessing

x = .values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pandas.DataFrame(x_scaled)